In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('boat.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Otsu thresholding
_, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through the contours and draw bounding boxes
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the original image with bounding boxes
cv2.imshow('Segmented Image with Bounding Boxes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

def apply_morphological_transformations(size):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    
    # Perform morphological transformations
    eroded = cv2.erode(thresholded, kernel)
    dilated = cv2.dilate(thresholded, kernel)
    opened = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)
    closed = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)
    
    # Concatenate images horizontally for display
    output = np.hstack((eroded, dilated, opened, closed))
    
    # Display the result
    cv2.imshow('Morphological Transformations', output)

# Load the image and convert it to grayscale
image = cv2.imread('leaf.jfif')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to segment the leaf
_, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Create a window and trackbar
cv2.namedWindow('Morphological Transformations')
cv2.createTrackbar('Structuring Element Size', 'Morphological Transformations', 1, 30, apply_morphological_transformations)

# Initialize with the initial size
apply_morphological_transformations(1)

# Wait for a key press to exit
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def color_seg(img, color_low, color_high, smooth=True):
    """Args:
    img: source image RGB,
    color_low and color_high are tuples of color in HSV,
    smooth: boolean which control whether image will be blurred"""
    img_ori = img.copy()
    if smooth:
        img = cv.GaussianBlur(img, (5, 5), 0)
        
    # Change color space
    img_hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)
    
    # Mask and result
    mask = cv.inRange(img_hsv, color_low, color_high)
    # Morphological operations
    mask = cv.erode(mask, None, iterations=2)
    mask = cv.dilate(mask, None, iterations=2)
    return cv.bitwise_and(img_ori, img_ori, mask=mask)

# Define the blue color range in HSV
blue_low = np.array([90, 50, 50])
blue_high = np.array([130, 255, 255])

# Load and process the porcelain images
porcelain_imgs = []
for i in range(1, 6):
    img = cv.imread(cv.samples.findFile(f"images/color_space/porcelain{i}.jfif"))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    porcelain_imgs.append(img)
    
# Apply color segmentation to porcelain images
porcelain_segmented = [color_seg(im, blue_low, blue_high) for im in porcelain_imgs]

# Display the original porcelain images and their segmented results
fig, ax = plt.subplots(5, 2, figsize=(10, 12))

for i in range(5):
    ax[i][0].imshow(porcelain_imgs[i])
    ax[i][0].axis("off")
    ax[i][1].imshow(porcelain_segmented[i])
    ax[i][1].axis('off')
    
fig.suptitle("Porcelain Images and Their Segmentations")
plt.tight_layout()
plt.show()